<a target="_blank" href="https://colab.research.google.com/github/Jasonchen0317/CSGY-6613-Assignment/blob/main/assignment-3/assignment-3.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [2]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00


In [7]:
from pytube import YouTube
import cv2
import os

In [4]:
def download_video(video_list):
    for i in range(len(video_list)):
        #get link
        link = video_list[i]
        yt = YouTube(link)
        yt.bypass_age_gate()
        #Download video
        filename = "%s.mp4" % str(i)
        yt.streams.filter(res="720p").first().download(filename=filename, output_path='video')

In [5]:
if not os.path.exists('video'):
   os.makedirs('video')

video_list = [
    'https://www.youtube.com/watch?v=DhmZ6W1UAv4',
    'https://www.youtube.com/watch?v=YrydHPwRelI'
]

In [6]:
download_video(video_list)

In [8]:
if not os.path.exists('frames'):
   os.makedirs('frames')

In [9]:
v_list = os.listdir('video')

In [11]:
len(v_list)

2

In [15]:
for i in range(len(v_list)):
  vid = i
  capture = cv2.VideoCapture(f'video/{vid}.mp4')
  frameNr = 0
  success, frame = capture.read()
  while (success):
      cv2.imwrite(f'frames/video_{vid}_frame_{frameNr}.jpg', frame)

      frameNr = frameNr+1

  capture.release()


Streaming output truncated to the last 5000 lines.
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

KeyboardInterrupt: ignored

In [ ]:
f_list = os.listdir('frames')
print(len(f_list))